<a href="https://colab.research.google.com/github/liaoqijia/WVS7/blob/main/xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

qijia91_wvs_imputed_path = kagglehub.dataset_download('qijia91/wvs-imputed')

print('Data source import complete.')


# XGBoost Model

In [ ]:
pip install pyreadr

In [ ]:
import pyreadr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error
from xgboost import XGBRegressor
import optuna

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

path = '/kaggle/input/wvs-imputed/impute_test.rds'
result = pyreadr.read_r(path)

In [ ]:
df = result[None] # extract the pandas data frame
print('data shape', df.shape)
print('data shape', df.info())

In [ ]:
train = df.iloc[:65648,:]
test = df.iloc[65648:,:]

print(f"train shape:", {train.shape})
print(f"test shape:", {test.shape})

X_train = train.drop(['activism'], axis=1)
X_test = test.drop(['activism'], axis=1)

y_train = train['activism']
y_test = test['activism']

print(f"X_train shape:", {X_train.shape})
print(f"X_test shape:", {X_test.shape})
print(f"y_train shape:", {y_train.shape})
print(f"y_test shape:", {y_test.shape})

In [ ]:
for col in X_train.columns:
    print(col)

## XGBoost Baseline model

In [ ]:
# Define the model
xgb_untuned = XGBRegressor(tree_method="hist", enable_categorical=True, device="cuda", max_cat_to_onehot=1,verbosity = 2, n_jobs = -1)

# Evaluate the model using cross-validation
scores = cross_val_score(xgb_untuned, X_train, y_train, cv=5, scoring = "neg_mean_squared_error")

In [ ]:
# Fit the model
xgb_untuned.fit(X_train, y_train)

In [ ]:
# Make predictions
y_pred = xgb_untuned.predict(X_test)

MSE_test = mean_squared_error(y_test, y_pred)
RMSE_test = mean_squared_error(y_test, y_pred)**0.5
R2_test = r2_score(y_test, y_pred)
# Evaluate the model
print('MSE: ', np.round(MSE_test, 4))
print('RMSE: ', np.round(RMSE_test, 4))
print('R2: ', np.round(R2_test, 4))


In [ ]:
df_xgb_untuned = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
correlation_matrix = df_xgb_untuned.corr()
print(correlation_matrix)

## XGBoost Tuning with 50 trials

In [ ]:
# Use Optuna to tune the XGBRegressor model

def objective(trial):
    param = {
        'max_depth': trial.suggest_int('max_depth', 1, 20),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.5),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0.01, 1.0),
        'subsample': trial.suggest_float('subsample', 0.01, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.01, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.01, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.01, 1.0)
    }
    xgb_tuned = XGBRegressor(**param, tree_method="hist", enable_categorical=True, max_cat_to_onehot=1, device="cuda", verbosity = 2,  random_state=10, n_jobs = -1)
    scores = cross_val_score(xgb_tuned, X_train, y_train, cv=5, scoring = "neg_mean_squared_error")
    xgb_tuned.set_params(early_stopping_rounds=10)


    xgb_tuned.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)
    y_pred = xgb_tuned.predict(X_test)
    MSE = (mean_squared_error(y_test, y_pred))

    return np.round(MSE, 3)

In [ ]:
# Create the study
sampler = optuna.samplers.RandomSampler(seed = 42)
study = optuna.create_study(direction='minimize', study_name='xgboost_regression', sampler=sampler)
study.optimize(objective, n_trials=50,show_progress_bar=True)

In [ ]:
# Print the best parameters
print('Best parameters', study.best_params)
print('Best value', study.best_value)
print('Best trial', study.best_trial)

In [ ]:
# Best parameters {'max_depth': 7, 'learning_rate': 0.05762328709905395, 'n_estimators': 929, 'min_child_weight': 9, 'gamma': 0.26536221143800404, 'subsample': 0.6633842055738373, 'colsample_bytree': 0.8190499781992037, 'reg_alpha': 0.5596488034834677, 'reg_lambda': 0.5343540725724464}
study_best_params = {'max_depth': 7, 'learning_rate': 0.05762328709905395, 'n_estimators': 929, 'min_child_weight': 9, 'gamma': 0.26536221143800404, 'subsample': 0.6633842055738373, 'colsample_bytree': 0.8190499781992037, 'reg_alpha': 0.5596488034834677, 'reg_lambda': 0.5343540725724464}


In [ ]:
# Use the optimal hyperparameters to train the model

xgb_tuned_final = XGBRegressor(**study_best_params, tree_method="hist", enable_categorical=True, max_cat_to_onehot=1, device="cuda", verbosity = 2)
xgb_tuned_final.fit(X_train, y_train)
y_pred = xgb_tuned_final.predict(X_test)

print('MSE: ', mean_squared_error(y_test, y_pred))
print('RMSE: ', np.sqrt(mean_squared_error(y_test, y_pred)))
print('R2: ', r2_score(y_test, y_pred))

In [ ]:
df_xgb_tuned = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
correlation_matrix = df_xgb_tuned.corr()
print(correlation_matrix)

In [ ]:
# Plot the first set of parameters
fig1 = optuna.visualization.matplotlib.plot_slice(study, params=["n_estimators", "max_depth", "learning_rate", "min_child_weight"])
plt.show()

# Plot the second set of parameters
fig2 = optuna.visualization.matplotlib.plot_slice(study, params=["gamma", "subsample", "colsample_bytree", "reg_alpha"])
plt.show()


In [ ]:
import matplotlib.pyplot as plt
optuna.visualization.plot_optimization_history(study)
optuna.visualization.plot_parallel_coordinate(study, params=["n_estimators", "max_depth", "learning_rate", "min_child_weight", "gamma", "subsample", "colsample_bytree", "reg_alpha", "reg_lambda" ])
optuna.visualization.plot_param_importances(study)

# Regression models

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import Lasso, Ridge, ElasticNet

# determine categorical and numerical features
numerical_features_train = X_train.select_dtypes(include=['float64']).columns
categorical_features_train = X_train.select_dtypes(include=['category']).columns

# Define the preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features_train),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features_train)
    ]
)


## baseline ridge model

In [ ]:
# define the Ridge model
model_Ridge = Ridge(random_state = 2)
pipeline_Ridge = Pipeline(steps=[('prep',preprocessor), ('m', model_Ridge)])
scores = cross_val_score(pipeline_Ridge, X_train, y_train, scoring='neg_mean_squared_error', cv=5, n_jobs = -1)

RMSE_ridge = (-scores)**0.5
print("RMSE_ridge", RMSE_ridge)
print("Mean RMSE_ridge", RMSE_ridge.mean())

## Ridge Regression with Tuning

In [ ]:
# Define the model (Ridge)

def objective(trial):
    alpha = trial.suggest_float('alpha', 1e-4, 10.0, log=True)  # Ridge regression alpha parameter
    model_Ridge = Ridge(alpha=alpha, random_state = 2)
    pipeline_Ridge = Pipeline(steps=[('prep', preprocessor), ('m', model_Ridge)])
    scores = cross_val_score(pipeline_Ridge, X_train, y_train, scoring='neg_mean_squared_error', cv=5, n_jobs= -1)
    rmse_ridge = (-scores)**0.5
    return rmse_ridge.mean()

# Perform hyperparameter optimization with Optuna
sampler = optuna.samplers.RandomSampler(seed = 42)
study_Ridge = optuna.create_study(direction='minimize', study_name='ridge_regression', sampler=sampler)
study_Ridge.optimize(objective, n_trials=50, show_progress_bar= True)

In [ ]:
# Get the best hyperparameters and best score
best_alpha_Ridge = study_Ridge.best_params
best_score_Ridge = study_Ridge.best_value

print("Best alpha:", best_alpha_Ridge)
print("Best score:", best_score_Ridge)

## last fit the best model of ridge regression

In [ ]:
Best_alpha = {'alpha': 7.072114131472227}

# Create a pipeline with preprocessing and model
pipeline_Ridge = Pipeline(steps=[('prep', preprocessor), ('m', Ridge(**Best_alpha, random_state=2))])

# Fit the pipeline on training data
pipeline_Ridge.fit(X_train, y_train)

# Make predictions on testing data
y_pred = pipeline_Ridge.predict(X_test)

# Evaluate the predictions
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print('Ridge_MSE Test: ', mse)
print('Ridge_RMSE Test: ', rmse)
print('Ridge_R2 Test: ', r2)

In [ ]:
df_Ridge = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
correlation_matrix = df_Ridge.corr()
print(correlation_matrix)

## run baseline lasso model

In [ ]:
%%time
# define the lasso model
model_lasso = Lasso(random_state = 2)
# define the data preparation and modeling pipeline
pipeline_lasso = Pipeline(steps=[('prep',preprocessor), ('m', model_lasso)])

scores = cross_val_score(pipeline_lasso, X_train, y_train, scoring='neg_mean_squared_error', cv=5, n_jobs=-1)


RMSE_lasso = (-scores)**0.5
print("RMSE_lasso", RMSE_lasso)
print("Mean RMSE_lasso", RMSE_lasso.mean())

# Lasso Regression with Tuning

In [ ]:
def objective(trial):
    # Define the hyperparameters to tune
    alpha = trial.suggest_float('alpha', 1e-5, 10.0, log=True)  # Larger range for alpha

    # Create the model
    model_lasso = Lasso(alpha=alpha,random_state=2)
    pipeline_lasso = Pipeline(steps=[('prep', preprocessor), ('m', model_lasso)])
    scores = cross_val_score(pipeline_lasso, X_train, y_train, scoring='neg_mean_squared_error', cv=5, n_jobs = -1)
    rmse_lasso = (-scores)**0.5
    return rmse_lasso.mean()

In [ ]:
# Perform hyperparameter optimization with Optuna
sampler = optuna.samplers.RandomSampler(seed = 42)
study_lasso = optuna.create_study(direction='minimize', study_name='lasso_regression', sampler=sampler)
study_lasso.optimize(objective, n_trials=50, show_progress_bar=True)

# Get the best hyperparameters and best score
best_params_lasso = study_lasso.best_params
best_value_lasso = study_lasso.best_value

print("Lasso Best alpha:", best_params_lasso)
print("Lasso Best score:", best_value_lasso)

In [ ]:
best_params_lasso

In [ ]:
best_params_lasso = {'alpha': 0.00015777663630582464}

In [ ]:
# Create a pipeline with preprocessing and model
pipeline_lasso = Pipeline(steps=[('prep', preprocessor), ('m', Lasso(**best_params_lasso, random_state=2))])

# Fit the pipeline on training data
pipeline_lasso.fit(X_train, y_train)

# Make predictions on testing data
y_pred = pipeline_lasso.predict(X_test)

# Evaluate the predictions
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print('Lasso MSE Test: ', mse)
print('Lasso RMSE Test: ', rmse)
print('Lasso R2 Test: ', r2)

In [ ]:
df_lasso = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
correlation_matrix = df_lasso.corr()
print(correlation_matrix)


# plot the performance of modesl on unseen data

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt

sns.set_style("whitegrid")

f, axes = plt.subplots(2, 2, figsize = (12,10))


sns.regplot(data=df_xgb_untuned, x='Actual', y='Predicted', scatter_kws={'s': 5}, ax=axes[0,0])
axes[0,0].set_title("XGB Regression (defult)")
axes[0,0].text(0.1, 2, 'R = 0.732', fontsize=10, color='red')
axes[0,0].text(0.1, 1.9, 'RMSE = 0.359', fontsize=10, color='red')
axes[0,0].text(0.1, 1.8, 'R2 = 0.534', fontsize=10, color='red')


sns.regplot(data=df_xgb_tuned, x='Actual', y='Predicted', scatter_kws={'s': 5}, ax=axes[0,1])
axes[0,1].set_title("Fine Tuned XGB Regression")
axes[0,1].text(0.1, 2, 'R = 0.755', fontsize=10, color='red')
axes[0,1].text(0.1, 1.9, 'RMSE = 0.344', fontsize=10, color='red')
axes[0,1].text(0.1, 1.8, 'R2 = 0.571', fontsize=10, color='red')

sns.regplot(data=df_Ridge, x='Actual', y='Predicted', scatter_kws={'s': 5}, ax=axes[1,0])
axes[1,0].set_title("Fine Tuned Ridge Regression")
axes[1,0].text(0.1, 2, 'R = 0.684', fontsize=10, color='red')
axes[1,0].text(0.1, 1.9, 'RMSE = 0.383', fontsize=10, color='red')
axes[1,0].text(0.1, 1.8, 'R2 = 0.468', fontsize=10, color='red')

sns.regplot(data=df_lasso, x='Actual', y='Predicted', scatter_kws={'s': 5}, ax=axes[1,1])
axes[1,1].set_title("Fine Tuned Lasso Regression")
axes[1,1].text(0.1, 2, 'R = 0.684', fontsize=10, color='red')
axes[1,1].text(0.1, 1.9, 'RMSE = 0.383', fontsize=10, color='red')
axes[1,1].text(0.1, 1.8, 'R2 = 0.468', fontsize=10, color='red')

# model interpretation

In [ ]:
pip install dalex

## create the explainer

In [ ]:
import dalex as dx

xgb_untuned_explainer = dx.Explainer(xgb_untuned, X_test, y_test, label="xgb untuned")
xgb_tuned_explainer = dx.Explainer(xgb_tuned_final, X_test, y_test, label="xgb fine tuned")

Ridge_tuned_explainer = dx.Explainer(pipeline_Ridge, X_test, y_test, label="ridge fine tuned")
Lasso_tuned_explainer = dx.Explainer(pipeline_lasso, X_test, y_test, label="lasso fine tuned")

xgb_untuned_perform = xgb_untuned_explainer.model_performance().result
xgb_tuned_perform = xgb_tuned_explainer.model_performance().result
Ridge_tuned_perform = Ridge_tuned_explainer.model_performance().result
Lasso_tuned_perform = Lasso_tuned_explainer.model_performance().result


## model performance evaluation: residuals

In [ ]:
exp_list = [xgb_untuned_explainer, xgb_tuned_explainer, Ridge_tuned_explainer, Lasso_tuned_explainer]
exp_list[0].model_performance().plot([exp.model_performance() for exp in exp_list[1:]])


## model surogate: decision tree approximation

In [ ]:
xgb_tuned_explainer.model_surrogate().plot()

## Partial Dependence Plot of Cultural Dimensions

In [ ]:
exp_list[0].model_parts(loss_function='rmse', random_state =0).plot([exp.model_parts(loss_function='rmse',random_state =0) for exp in exp_list[1:]])

In [ ]:
exp_list[0].model_profile(variable_splits_type="quantiles").plot(
    [exp.model_profile(variable_splits_type="quantiles") for exp in exp_list[1:]],
    variables =['IND', 'LTO', 'PD', 'MASC', 'UA'],
    title="Partial Dependence"
)

* Q199. How interested would you say you are in politics?
* Q200. When you get together with your friends, would you say you discuss political matters frequently, occasionally or never?
* Q206. (Internet) please indicate whether you use it to obtain information daily, weekly, monthly, less than monthly or never
* Q208. (Talk with friends or colleagues), People learn what is going on in this country and the world from various sources.
* Q222. (National level) When elections take place, do you vote always, usually or never? Please tell me separately for each of the following levels
* Q227. (Voters are bribed), In your view, how often do the following things occur in this country’s elections?

In [ ]:
exp_list[0].model_profile(variable_splits_type="quantiles").plot(
    [exp.model_profile(variable_splits_type="quantiles") for exp in exp_list[1:]],
    variables =['Q199_Political_Interest_Participation',
                'Q200_Political_Interest_Participation',
                'Q206_Political_Interest_Participation',
                'Q222_Political_Interest_Participation',
                'Q208_Political_Interest_Participation',
                'Q227_Political_Interest_Participation'],
    title="Partial Dependence"
)

* Q4-Politics: For each of the following, indicate how important it is in your life.
* Q41 Work should always come first, even if it means less spare time
* Q148 To what degree are you worried about the following situations?

In [ ]:
exp_list[0].model_profile(variable_splits_type="quantiles").plot(
    [exp.model_profile(variable_splits_type="quantiles") for exp in exp_list[1:]],
    variables =['Q4_Social_Values_Norms_Stereotypes',
                'Q41_Social_Values_Norms_Stereotypes',
                'Q148_Perception_Security'],
    title="Partial Dependence"
)

* Q98 Political party: Now I am going to read off a list of voluntary organizations. For each organization, could you tell me whether you are an active member, an inactive member or not a member of that type of organization?

In [ ]:

exp_list[0].model_profile(variable_splits_type="quantiles", variable_type= 'categorical').plot(
    [exp.model_profile(variable_splits_type="quantiles", variable_type= 'categorical') for exp in exp_list[1:]],
    variables =['Q98R_SocialCapital_Trust_Orga_Member'],
    title="Partial Dependence"
)